In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

#import tf_keras as keras_legacy
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers import TextVectorization

import local_lib.lib_tf as lt
import local_lib.text_helpers as th
import local_lib.savers as lsv

# import sys
# LIBRARY_PATH = '/Users/thomasbayer/Projects/PycharmProjects/support/ml/src/'
# sys.path.append(LIBRARY_PATH)
# import lib_tf as lt
# import lib.savers as sv
from tensorflow.keras.models import load_model

lt.init_gpu()
#%load_ext tensorboard


2024-08-29 09:45:03.491194: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2024-08-29 09:45:03.491214: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-08-29 09:45:03.491218: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-08-29 09:45:03.491248: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-29 09:45:03.491262: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Tensorflow v 2.15.0
Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
setting memory growth for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') to True
Physical devices cannot be modified after being initialized


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Loading the Data and Preparing the Dataset
Dataset for character-based predictions

In [2]:
ROOT_DIRECTORY = "/Users/thomasbayer/data/"
MODEL_ROOT = os.path.join(ROOT_DIRECTORY, "data_science/models/")
SEED = 42
model_saver = lsv.ModelSaver(MODEL_ROOT)

filepath = os.path.join(ROOT_DIRECTORY, "data_science/text/shakespeare.txt")

with open(filepath) as f:
    shakespeare_text = f.read()

print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [22]:
np.random.seed(42)
tf.random.set_seed(42)

batch_size = 128
n_steps = 100
#n_steps = 75

tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(shakespeare_text)
config = tokenizer.get_config()

print(tokenizer.texts_to_sequences(["First"]))
print(tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]]))

max_id = len(tokenizer.word_index)  # number of distinct characters
dataset_size = tokenizer.document_count  # total number of characters

print("max_id = {}, dataset_size = {:,.0f}".format(max_id, dataset_size))

encoded_raw = tokenizer.texts_to_sequences([shakespeare_text])
print(f"encoded_raw {len(encoded_raw[0])}")
encoded = np.array(encoded_raw[0]) - 1
print(f"encoded.shape {encoded.shape}")
#[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
print(f"train_size = {train_size}")

dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
for X in dataset.take(7):
    print(f"X={X}, token={tokenizer.sequences_to_texts([[X.numpy() + 1]])}")

window_length = n_steps + 1  # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
for X in dataset.take(1):
    print(f"X={X.shape}")
dataset = dataset.shuffle(10000).batch(batch_size)
# for X in dataset.take(1):
#     print(f"X={X.shape}")
#     print(f"window={X[:, :-1].shape}")
#     print(f"target={X[:, 1:].shape}")
#     print(f"window={X[:, :-1]}")
#     print(f"window={X[:, 1:]}")

# generate X,y pairs
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

[[20, 6, 9, 8, 3]]
['f i r s t']
max_id = 39, dataset_size = 1,115,394
encoded_raw 1115394
encoded.shape (1115394,)
train_size = 1003854
X=19, token=['f']
X=5, token=['i']
X=8, token=['r']
X=7, token=['s']
X=2, token=['t']
X=0, token=[' ']
X=18, token=['c']
X=(101,)


In [23]:
import tensorflow_datasets as tfds

tfds.benchmark(dataset, batch_size=128, num_iter=100)


************ Summary ************



  0%|          | 0/100 [00:00<?, ?it/s]

Examples/sec (First included) 9375.63 ex/sec (total: 12928 ex, 1.38 sec)
Examples/sec (First only) 214.38 ex/sec (total: 128 ex, 0.60 sec)
Examples/sec (First excluded) 16371.85 ex/sec (total: 12800 ex, 0.78 sec)


,duration,num_examples,avg
first+lasts,1.378894,12928,9375.629748
first,0.597064,128,214.382318
lasts,0.781830,12800,16371.848159


In [4]:
for X, y in dataset.take(1):
    print("X.shape = ", X.shape)
    print("token = ", tokenizer.sequences_to_texts([X[1].numpy() + 1]))
    print("y.shape = ", y.shape)
    print("token = ", tokenizer.sequences_to_texts([y[1].numpy() + 1]))

X.shape =  (128, 100)
token =  ['h e   k i n g l y - c r o w n e d   h e a d ,   t h e   v i g i l a n t   e y e , \n t h e   c o u n s e l l o r   h e a r t ,   t h e   a r m   o u r   s o l d i e r , \n o u r   s t e e d   t h e   l']
y.shape =  (128, 100)
token =  ['e   k i n g l y - c r o w n e d   h e a d ,   t h e   v i g i l a n t   e y e , \n t h e   c o u n s e l l o r   h e a r t ,   t h e   a r m   o u r   s o l d i e r , \n o u r   s t e e d   t h e   l e']


In [5]:
dataset_byte = dataset.prefetch(tf.data.AUTOTUNE)
dataset = dataset.map(
        lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
        )
dataset = dataset.prefetch(tf.data.AUTOTUNE)

for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)
    print(X_batch[0, :, :])
    print(Y_batch[0, :])

(128, 100, 39) (128, 100)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(100, 39), dtype=float32)
tf.Tensor(
[ 0  2  6  5  7 29 10 10 14  1  9  1  9  5 13  7 23 10  2  6  1  0  7  1
  9  4  2  3  8  7  0  3 19  0  8  3 14  1  0  4  8  1  0  2  6  5  7  0
 20  3  3 12  0 21  1 11 11 15 17 10  4  9 12  0 15  3 13  0  2  6  1  0
 14 13  2  5  9  3 13  7  0 14  1 14 21  1  8  7 28  0 19  3  8  0  1 34
  4 14  5  9], shape=(100,), dtype=int64)


## Creating and Training the Model

In [6]:
model_name = "shakespeare_one_hot"
m1 = keras.models.Sequential(
        [
                keras.layers.GRU(
                        128, return_sequences=True, input_shape=[None, max_id],
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ], name="Single_Layer_GRU128"
        )
m1.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m1.summary()

Model: "Single_Layer_GRU128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 128)         64896     
                                                                 
 time_distributed (TimeDist  (None, None, 39)          5031      
 ributed)                                                        
                                                                 
Total params: 69927 (273.15 KB)
Trainable params: 69927 (273.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
m1b = keras.models.Sequential(
        [
                keras.layers.Bidirectional(
                        keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0),
                        merge_mode='sum'
                        ),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ], name="Single_Bi_GRU128"
        )
m1b.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
#m1b.summary()

In [8]:
m2b = keras.models.Sequential(
        [
                keras.layers.Bidirectional(
                        keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0),
                        merge_mode='ave'
                        ),
                keras.layers.GRU(
                        128, return_sequences=True,
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ], name="Single_Bi_GRU128_Uni_GRU128"
        )
m2b.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
#m2b.summary()


In [9]:
m2 = keras.models.Sequential(
        [
                keras.layers.GRU(
                        128, return_sequences=True, input_shape=[None, max_id],
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.GRU(
                        128, return_sequences=True,
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ],
        name="3L_2xGRU128_1xTimeDistributed"
        )
m2.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m2.summary()

Model: "3L_2xGRU128_1xTimeDistributed"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, None, 128)         64896     
                                                                 
 gru_5 (GRU)                 (None, None, 128)         99072     
                                                                 
 time_distributed_3 (TimeDi  (None, None, 39)          5031      
 stributed)                                                      
                                                                 
Total params: 168999 (660.15 KB)
Trainable params: 168999 (660.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
m3 = keras.models.Sequential(
        [
                keras.layers.GRU(
                        128, return_sequences=True, input_shape=[None, max_id],
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.GRU(
                        128, return_sequences=True,
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ], name="2_Layer_GRU_128"
        )
m3.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m3.summary()

Model: "2_Layer_GRU_128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_6 (GRU)                 (None, None, 128)         64896     
                                                                 
 gru_7 (GRU)                 (None, None, 128)         99072     
                                                                 
 time_distributed_4 (TimeDi  (None, None, 39)          5031      
 stributed)                                                      
                                                                 
Total params: 168999 (660.15 KB)
Trainable params: 168999 (660.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
m4 = keras.models.Sequential(
        [
                keras.layers.GRU(
                        256, return_sequences=True, batch_input_shape=[batch_size, n_steps, max_id],
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.GRU(
                        128, return_sequences=True, stateful=True,
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ]
        )
m4.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m4.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_8 (GRU)                 (128, 100, 256)           228096    
                                                                 
 gru_9 (GRU)                 (128, 100, 128)           148224    
                                                                 
 time_distributed_5 (TimeDi  (128, 100, 39)            5031      
 stributed)                                                      
                                                                 
Total params: 381351 (1.45 MB)
Trainable params: 381351 (1.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
m5 = keras.models.Sequential(
        [
                keras.layers.LSTM(
                        64, return_sequences=True, input_shape=[None, max_id],
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.LSTM(
                        64, return_sequences=True,
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ], name="2_Layer_LSTM_64"
        )
m5.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m5.summary()

Model: "2_Layer_LSTM_64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 64)          26624     
                                                                 
 lstm_1 (LSTM)               (None, None, 64)          33024     
                                                                 
 time_distributed_6 (TimeDi  (None, None, 39)          2535      
 stributed)                                                      
                                                                 
Total params: 62183 (242.90 KB)
Trainable params: 62183 (242.90 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
m6 = keras.models.Sequential(
        [
                keras.layers.GRU(
                        128, return_sequences=True, input_shape=[None, max_id],
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.GRU(
                        128, return_sequences=True,
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.GRU(
                        128, return_sequences=True,
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ], name="3_Layer_GRU_128"
        )
m6.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m6.summary()

Model: "3_Layer_GRU_128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_10 (GRU)                (None, None, 128)         64896     
                                                                 
 gru_11 (GRU)                (None, None, 128)         99072     
                                                                 
 gru_12 (GRU)                (None, None, 128)         99072     
                                                                 
 time_distributed_7 (TimeDi  (None, None, 39)          5031      
 stributed)                                                      
                                                                 
Total params: 268071 (1.02 MB)
Trainable params: 268071 (1.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
m7 = keras.models.Sequential(
        [
                keras.layers.GRU(
                        64, return_sequences=True, input_shape=[None, max_id],
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.GRU(
                        64, return_sequences=True,
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.GRU(
                        64, return_sequences=True,
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.GRU(
                        64, return_sequences=True,
                        dropout=0, recurrent_dropout=0
                        ),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ], name="4_Layer_GRU_64"
        )
m7.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m7.summary()


Model: "4_Layer_GRU_64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_13 (GRU)                (None, None, 64)          20160     
                                                                 
 gru_14 (GRU)                (None, None, 64)          24960     
                                                                 
 gru_15 (GRU)                (None, None, 64)          24960     
                                                                 
 gru_16 (GRU)                (None, None, 64)          24960     
                                                                 
 time_distributed_8 (TimeDi  (None, None, 39)          2535      
 stributed)                                                      
                                                                 
Total params: 97575 (381.15 KB)
Trainable params: 97575 (381.15 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [15]:
LOSS_THRESHOLD = 0.1


class OnThresholdStop(tf.keras.callbacks.Callback):
    def __init__(self, threshold=LOSS_THRESHOLD):
        self.threshold = threshold

    def on_train_batch_end(self, epoch, logs={}):
        current_loss = logs.get('loss')
        if current_loss < self.threshold:
            print("\n Reached loss = {:.3f}, so stopping training!!".format(current_loss))
            self.model.stop_training = True


early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        min_delta=0.005, patience=1,
        mode='auto',
        verbose=True,
        restore_best_weights=True
        )

In [17]:
%%time
model = m1
epochs = 5

steps_per_epoch = train_size // batch_size

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 4.05 μs


In [18]:
loss_threshold = OnThresholdStop(0.1)
print("steps per epoch = {:,.0f}".format(steps_per_epoch))
history = lt.eval_model_ds(
        model,
        dataset, steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        callbacks=[early_stopping, loss_threshold],
        save=True,
        )

#print('Loss: {:.3f}'.format(history.history['loss'][-1]))
#model_saver.save(model, model_name + '-' + model.name)

steps per epoch = 7,842
train model Single_Layer_GRU128, batch_size = 128
Epoch 1/5


2024-08-28 10:43:13.661460: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


7842/7842 [==============================] - 143s 18ms/step - loss: 1.4903
Epoch 2/5
7842/7842 [==============================] - 140s 18ms/step - loss: 1.3093
Epoch 3/5
7842/7842 [==============================] - 143s 18ms/step - loss: 1.2911
Epoch 4/5
7842/7842 [==============================] - 140s 18ms/step - loss: 1.2810
Epoch 5/5
7842/7842 [==============================] - 141s 18ms/step - loss: 1.2728
Time: 707.07 seconds


In [19]:
model = m1b
history = lt.eval_model_ds(
        model,
        dataset, steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        callbacks=[early_stopping, loss_threshold],
        save=True,
        )

train model Single_Bi_GRU128, batch_size = 128
Epoch 1/5
3334/7842 [===========>..................] - ETA: 2:25 - loss: 0.1000
 Reached loss = 0.100, so stopping training!!
7842/7842 [==============================] - 109s 14ms/step - loss: 0.1000
Time: 109.46 seconds


In [33]:
loss_threshold = OnThresholdStop(0.1)
print("steps per epoch = {:,.0f}".format(steps_per_epoch))
history = model.fit(
        dataset, steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        callbacks=[early_stopping, loss_threshold]
        )

print('Loss: {:.3f}'.format(history.history['loss'][-1]))
model_saver.save(model, model_name + '-' + model.name)

steps per epoch = 7,842
Epoch 1/2


2024-08-26 12:57:28.714484: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 245/7842 ━━━━━━━━━━━━━━━━━━━━ 3:40:41 2s/step - loss: 3.0351

KeyboardInterrupt: 

In [24]:
model = model_saver.load(name="shakespeare_one_hot-3L_2xGRU128_1xTimeDistributed")

Model 3L_2xGRU128_1xTimeDistributed loaded from /Users/thomasbayer/data/data_science/models/models/shakespeare_one_hot-3L_2xGRU128_1xTimeDistributed


In [19]:
ms = lt.ModelSaver("fake_text_1")
loss_threshold = OnThresholdStop(0.8)
epochs = 4
steps_per_epoch = train_size // batch_size
print("steps per epoch = {:,.0f}".format(steps_per_epoch))
model = m1b
history = lt.run_ds(
        model, dataset, epochs=epochs, train_size=train_size, model_saver=ms,
        callbacks=[early_stopping, loss_threshold], verbose=True
        )

steps per epoch = 7,842
--- Model Single_Bi_GRU128 ---
Weights will be saved!
batch_size = 128
steps per epoch = 7,842
Epoch 1/4
 319/7842 [>.............................] - ETA: 4:06 - loss: 0.7982    

In [16]:
model = model_saver.load(name="shakespeare_100_128_128_39")

Model sequential loaded from /Users/thomasbayer/data/data_science/models/models/shakespeare_100_128_128_39


In [34]:
ms.save_weights(model)

In [19]:
model = m1
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#loss_object = tf.keras.losses.MeanAbsoluteError()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

with tf.GradientTape() as tape:
    for X_batch, y_batch in dataset.take(1):
        print(X_batch.shape)
        print(X_batch[0, :10, :])
        print("-- predictions --")
        predictions = model(X_batch, training=True)
        print(predictions.shape)
        print(predictions[0, :10])
        print("-- y --")
        print(y_batch.shape)
        print(y_batch[0, :10])
        loss = loss_object(y_batch, predictions)
        print("loss = {:.4f}".format(loss))

print("-- loss --")
print(train_loss(loss))
print(train_accuracy(y_batch, predictions))

Loss: 1.306


In [ ]:
X2 = X_batch[0, 1, :]
print(X2)
print(np.argmax(X2.numpy()))
y2 = y_batch[0, 0]
print(y2)
print(loss_object(y2, X2).numpy())

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
lo2 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
pred_1 = tf.constant([36, 41, 31, 0, 42], tf.int64)

In [ ]:
Xdata = X_batch[0, :5, :]
pred_2 = tf.constant([np.argmax(pred.numpy()) for pred in Xdata])
print(pred_2)
print(loss_object(pred_2, Xdata).numpy())
print(lo2(pred_2, Xdata).numpy())

## Using the Model to Generate Text

In [20]:
def preprocess(tokenizer, max_id, texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)


def next_char(text, model, tokenizer, max_id, temperature=1):
    X_new = preprocess(tokenizer, max_id, [text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]


def complete_text(text, n_chars, model, tokenizer, max_id, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, model, tokenizer, max_id, temperature)
    return text


def pnc(text):
    X_new = preprocess([text])
    Y_pred = model.predict_classes(X_new)
    last_char = tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]  # 1s
    print(text + last_char)


In [25]:
X_new = preprocess(tokenizer, max_id, ["How are yo"])
print(X_new.shape)
Y_pred = model.predict(X_new)
print(Y_pred.shape)
print(Y_pred)

(1, 10, 39)


2024-01-22 14:45:54.573251: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step
(1, 10, 39)
[[[1.20931156e-01 3.70303899e-01 2.02018861e-02 1.11474171e-01
   1.31978810e-01 1.50693133e-01 2.61593668e-04 1.28609617e-03
   3.13214911e-03 3.24410757e-06 3.01066786e-03 7.21530814e-05
   2.09957616e-05 1.90314278e-02 2.04755086e-03 2.97060404e-02
   4.09445929e-04 1.30923968e-02 6.21259504e-04 1.21234264e-03
   1.85215409e-04 4.04573092e-03 9.33994306e-04 1.47788238e-03
   1.66854981e-04 6.75439253e-04 1.89436576e-03 5.75556373e-03
   9.60931589e-04 7.16108771e-04 1.32984691e-03 2.26952392e-03
   2.30267433e-05 2.36209653e-05 1.51416862e-05 2.57177016e-05
   8.32794740e-06 1.82470603e-06 5.97795918e-07]
  [2.66175088e-03 1.11927157e-02 8.38375825e-04 1.81526113e-02
   5.22523792e-03 1.77089113e-03 2.92304358e-05 8.86492133e-02
   2.48556241e-01 2.74402648e-02 2.14733926e-04 1.53648689e-01
   9.91654349e-04 2.58494109e-01 9.39386413e-02 9.10922681e-05
   7.23988935e-02 5.41440444e-04 3.00254818e-04 3.11611133e-04
   1.31

In [26]:
one_hot_to_number = lambda x: np.argmax(x)
print([one_hot_to_number(x) for x in X_new[0, :, :]])
[one_hot_to_number(x) for x in Y_pred[0, :, :]]

[6, 3, 16, 0, 4, 8, 1, 0, 15, 3]


[1, 13, 0, 6, 18, 2, 0, 2, 3, 13]

In [27]:
rr = tokenizer.sequences_to_texts(Y_pred + 1)
print(len(rr[0]))  # 1st sentence, last char
print(rr[0][-1])

TypeError: unhashable type: 'numpy.ndarray'

In [23]:
#model = m5
complete_text("my ", 150, model=m1, tokenizer=tokenizer, max_id=max_id, temperature=0.9)

1/1 [==============================] - 0s 13ms/step


"my rancantio:\nhas our heart sooth, who is which i'll fellow is once must end,\nfor you, they are\nit pethe the rasciling in a that is the worshour be ride:"

In [19]:
pnc("h")

TypeError: preprocess() missing 2 required positional arguments: 'max_id' and 'texts'

In [ ]:
tf.random.set_seed(42)
ll = [np.log(0.5), np.log(0.4), np.log(0.1)]
print(ll)
print(tf.random.categorical([ll], num_samples=40).numpy())
print(tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy())

In [19]:
tf.random.set_seed(42)

next_char("How are yo", model, tokenizer, max_id, temperature=1)

ValueError: in user code:

    File "/Users/thomasbayer/Projects/PycharmProjects/nlp_1/venv311/lib/python3.11/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/Users/thomasbayer/Projects/PycharmProjects/nlp_1/venv311/lib/python3.11/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/thomasbayer/Projects/PycharmProjects/nlp_1/venv311/lib/python3.11/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/thomasbayer/Projects/PycharmProjects/nlp_1/venv311/lib/python3.11/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/Users/thomasbayer/Projects/PycharmProjects/nlp_1/venv311/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70,

In [ ]:
tf.random.set_seed(42)

print(complete_text("t", temperature=0.2))

In [ ]:
print(complete_text("t", n_chars=100, temperature=1))

In [ ]:
print(complete_text("r", temperature=0.8))

## Stateful RNN

In [ ]:
tf.random.set_seed(42)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
        lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
        )
dataset = dataset.prefetch(4)

In [ ]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
        lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
        )
dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential(
        [
                keras.layers.GRU(
                        128, return_sequences=True, stateful=True,
                        dropout=0.2, recurrent_dropout=0.2,
                        batch_input_shape=[batch_size, None, max_id]
                        ),
                keras.layers.GRU(
                        128, return_sequences=True, stateful=True,
                        dropout=0.2, recurrent_dropout=0.2
                        ),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ]
        )

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
history = model.fit(
        dataset, steps_per_epoch=steps_per_epoch, epochs=50,
        callbacks=[ResetStatesCallback()]
        )

To use the model with different batch sizes, we need to create a stateless copy. We can get rid of dropout since it is only used during training:

In [ ]:
stateless_model = keras.models.Sequential(
        [
                keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
                keras.layers.GRU(128, return_sequences=True),
                keras.layers.TimeDistributed(
                        keras.layers.Dense(
                                max_id,
                                activation="softmax"
                                )
                        )
                ]
        )

To set the weights, we first need to build the model (so the weights get created):

In [ ]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [ ]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [ ]:
tf.random.set_seed(42)

print(complete_text("t"))

# Sentiment Analysis

In [ ]:
tf.random.set_seed(42)

You can load the IMDB dataset easily:

In [ ]:
(X_train, y_test), (X_valid, y_test) = keras.datasets.imdb.load_data()

In [ ]:
X_train[0][:10]

In [ ]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

In [ ]:
datasets.keys()

In [ ]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [ ]:
train_size, test_size

In [ ]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

In [ ]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [ ]:
preprocess(X_batch, y_batch)

In [ ]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [ ]:
vocabulary.most_common()[:3]

In [ ]:
len(vocabulary)

In [ ]:
vocab_size = 10000
truncated_vocabulary = [
        word for word, count in vocabulary.most_common()[:vocab_size]]

In [ ]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

In [ ]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [ ]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

In [ ]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch


train_set = datasets["train"].repeat().batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [ ]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

In [ ]:
embed_size = 128
model = keras.models.Sequential(
        [
                keras.layers.Embedding(
                        vocab_size + num_oov_buckets, embed_size,
                        mask_zero=True,  # not shown in the book
                        input_shape=[None]
                        ),
                keras.layers.GRU(128, return_sequences=True),
                keras.layers.GRU(128),
                keras.layers.Dense(1, activation="sigmoid")
                ]
        )
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=5)

Or using manual masking:

In [ ]:
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=5)

## Reusing Pretrained Embeddings

In [ ]:
tf.random.set_seed(42)

In [ ]:
TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

In [ ]:
import tensorflow_hub as hub

model = keras.Sequential(
        [
                hub.KerasLayer(
                        "https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                        dtype=tf.string, input_shape=[], output_shape=[50]
                        ),
                keras.layers.Dense(128, activation="relu"),
                keras.layers.Dense(1, activation="sigmoid")
                ]
        )
model.compile(
        loss="binary_crossentropy", optimizer="adam",
        metrics=["accuracy"]
        )

In [ ]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].repeat().batch(batch_size).prefetch(1)
history = model.fit(train_set, steps_per_epoch=train_size // batch_size, epochs=5)

## Automatic Translation

In [ ]:
tf.random.set_seed(42)

In [ ]:
vocab_size = 100
embed_size = 10

In [ ]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
        decoder_cell, sampler,
        output_layer=output_layer
        )
final_outputs, final_state, final_sequence_lengths = decoder(
        decoder_embeddings, initial_state=encoder_state,
        sequence_length=sequence_lengths
        )
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
        inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
        outputs=[Y_proba]
        )

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
X = np.random.randint(100, size=10 * 1000).reshape(1000, 10)
Y = np.random.randint(100, size=15 * 1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

### Bidirectional Recurrent Layers

In [ ]:
model = keras.models.Sequential(
        [
                keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
                keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))
                ]
        )

model.summary()

### Positional Encoding

In [ ]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1:
            max_dims += 1  # max_dims must be even
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000 ** (2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000 ** (2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))

    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

In [ ]:
max_steps = 201
max_dims = 512
pos_emb = PositionalEncoding(max_steps, max_dims)
PE = pos_emb(np.zeros((1, max_steps, max_dims), np.float32))[0].numpy()

In [ ]:
i1, i2, crop_i = 100, 101, 150
p1, p2, p3 = 22, 60, 35
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(9, 5))
ax1.plot([p1, p1], [-1, 1], "k--", label="$p = {}$".format(p1))
ax1.plot([p2, p2], [-1, 1], "k--", label="$p = {}$".format(p2), alpha=0.5)
ax1.plot(p3, PE[p3, i1], "bx", label="$p = {}$".format(p3))
ax1.plot(PE[:, i1], "b-", label="$i = {}$".format(i1))
ax1.plot(PE[:, i2], "r-", label="$i = {}$".format(i2))
ax1.plot([p1, p2], [PE[p1, i1], PE[p2, i1]], "bo")
ax1.plot([p1, p2], [PE[p1, i2], PE[p2, i2]], "ro")
ax1.legend(loc="center right", fontsize=14, framealpha=0.95)
ax1.set_ylabel("$P_{(p,i)}$", rotation=0, fontsize=16)
ax1.grid(True, alpha=0.3)
ax1.hlines(0, 0, max_steps - 1, color="k", linewidth=1, alpha=0.3)
ax1.axis([0, max_steps - 1, -1, 1])
ax2.imshow(PE.T[:crop_i], cmap="gray", interpolation="bilinear", aspect="auto")
ax2.hlines(i1, 0, max_steps - 1, color="b")
cheat = 2  # need to raise the red line a bit, or else it hides the blue one
ax2.hlines(i2 + cheat, 0, max_steps - 1, color="r")
ax2.plot([p1, p1], [0, crop_i], "k--")
ax2.plot([p2, p2], [0, crop_i], "k--", alpha=0.5)
ax2.plot([p1, p2], [i2 + cheat, i2 + cheat], "ro")
ax2.plot([p1, p2], [i1, i1], "bo")
ax2.axis([0, max_steps - 1, 0, crop_i])
ax2.set_xlabel("$p$", fontsize=16)
ax2.set_ylabel("$i$", rotation=0, fontsize=16)
plt.savefig("positional_embedding_plot")
plt.show()

In [ ]:
embed_size = 512;
max_steps = 500;
vocab_size = 10000
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
positional_encoding = PositionalEncoding(max_steps, max_dims=embed_size)
encoder_in = positional_encoding(encoder_embeddings)
decoder_in = positional_encoding(decoder_embeddings)

Here is a (very) simplified Transformer (the actual architecture has skip connections, layer norm, dense nets, and most importantly it uses Multi-Head Attention instead of regular Attention):

In [ ]:
Z = encoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True)([Z, Z])

encoder_outputs = Z
Z = decoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True, causal=True)([Z, Z])
    Z = keras.layers.Attention(use_scale=True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(
        keras.layers.Dense(vocab_size, activation="softmax")
        )(Z)

Here's a basic implementation of the `MultiHeadAttention` layer. One will likely be added to `keras.layers` in the near future. Note that `Conv1D` layers with `kernel_size=1` (and the default `padding="valid"` and `strides=1`) is equivalent to a `TimeDistributed(Dense(...))` layer.

In [ ]:
K = keras.backend


class MultiHeadAttention(keras.layers.Layer):
    def __init__(self, n_heads, causal=False, use_scale=False, **kwargs):
        self.n_heads = n_heads
        self.causal = causal
        self.use_scale = use_scale
        super().__init__(**kwargs)

    def build(self, batch_input_shape):
        self.dims = batch_input_shape[0][-1]
        self.q_dims, self.v_dims, self.k_dims = [self.dims // self.n_heads] * 3  # could be hyperparameters instead
        self.q_linear = keras.layers.Conv1D(self.n_heads * self.q_dims, kernel_size=1, use_bias=False)
        self.v_linear = keras.layers.Conv1D(self.n_heads * self.v_dims, kernel_size=1, use_bias=False)
        self.k_linear = keras.layers.Conv1D(self.n_heads * self.k_dims, kernel_size=1, use_bias=False)
        self.attention = keras.layers.Attention(causal=self.causal, use_scale=self.use_scale)
        self.out_linear = keras.layers.Conv1D(self.dims, kernel_size=1, use_bias=False)
        super().build(batch_input_shape)

    def _multi_head_linear(self, inputs, linear):
        shape = K.concatenate([K.shape(inputs)[:-1], [self.n_heads, -1]])
        projected = K.reshape(linear(inputs), shape)
        perm = K.permute_dimensions(projected, [0, 2, 1, 3])
        return K.reshape(perm, [shape[0] * self.n_heads, shape[1], -1])

    def call(self, inputs):
        q = inputs[0]
        v = inputs[1]
        k = inputs[2] if len(inputs) > 2 else v
        shape = K.shape(q)
        q_proj = self._multi_head_linear(q, self.q_linear)
        v_proj = self._multi_head_linear(v, self.v_linear)
        k_proj = self._multi_head_linear(k, self.k_linear)
        multi_attended = self.attention([q_proj, v_proj, k_proj])
        shape_attended = K.shape(multi_attended)
        reshaped_attended = K.reshape(multi_attended, [shape[0], self.n_heads, shape_attended[1], shape_attended[2]])
        perm = K.permute_dimensions(reshaped_attended, [0, 2, 1, 3])
        concat = K.reshape(perm, [shape[0], shape_attended[1], -1])
        return self.out_linear(concat)

In [ ]:
Q = np.random.rand(2, 50, 512)
V = np.random.rand(2, 80, 512)
multi_attn = MultiHeadAttention(8)
multi_attn([Q, V]).shape

# Exercise solutions

## 1. to 7.

See Appendix A.

## 8.
_Exercise:_ Embedded Reber grammars _were used by Hochreiter and Schmidhuber in [their paper](https://homl.info/93) about LSTMs. They are artificial grammars that produce strings such as "BPBTSXXVPSEPE." Check out Jenny Orr's [nice introduction](https://homl.info/108) to this topic. Choose a particular embedded Reber grammar (such as the one represented on Jenny Orr's page), then train an RNN to identify whether a string respects that grammar or not. You will first need to write a function capable of generating a training batch containing about 50% strings that respect the grammar, and 50% that don't._

First we need to build a function that generates strings based on a grammar. The grammar will be represented as a list of possible transitions for each state. A transition specifies the string to output (or a grammar to generate it) and the next state.

In [ ]:
default_reber_grammar = [
        [("B", 1)],  # (state 0) =B=>(state 1)
        [("T", 2), ("P", 3)],  # (state 1) =T=>(state 2) or =P=>(state 3)
        [("S", 2), ("X", 4)],  # (state 2) =S=>(state 2) or =X=>(state 4)
        [("T", 3), ("V", 5)],  # and so on...
        [("X", 3), ("S", 6)],
        [("P", 4), ("V", 6)],
        [("E", None)]]  # (state 6) =E=>(terminal state)

embedded_reber_grammar = [
        [("B", 1)],
        [("T", 2), ("P", 3)],
        [(default_reber_grammar, 4)],
        [(default_reber_grammar, 5)],
        [("T", 6)],
        [("P", 6)],
        [("E", None)]]


def generate_string(grammar):
    state = 0
    output = []
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

Let's generate a few strings based on the default Reber grammar:

In [ ]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(default_reber_grammar), end=" ")

Looks good. Now let's generate a few strings based on the embedded Reber grammar:

In [ ]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(embedded_reber_grammar), end=" ")

Okay, now we need a function to generate strings that do not respect the grammar. We could generate a random string, but the task would be a bit too easy, so instead we will generate a string that respects the grammar, and we will corrupt it by changing just one character:

In [ ]:
POSSIBLE_CHARS = "BEPSTVX"


def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    return good_string[:index] + bad_char + good_string[index + 1:]

Let's look at a few corrupted strings:

In [ ]:
np.random.seed(42)

for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=" ")

We cannot feed strings directly to an RNN, so we need to encode them somehow. One option would be to one-hot encode each character. Another option is to use embeddings. Let's go for the second option (but since there are just a handful of characters, one-hot encoding would probably be a good option as well). For embeddings to work, we need to convert each string into a sequence of character IDs. Let's write a function for that, using each character's index in the string of possible characters "BEPSTVX":

In [ ]:
def string_to_ids(s, chars=POSSIBLE_CHARS):
    return [POSSIBLE_CHARS.index(c) for c in s]

In [ ]:
string_to_ids("BTTTXXVVETE")

We can now generate the dataset, with 50% good strings, and 50% bad strings:

In [ ]:
def generate_dataset(size):
    good_strings = [string_to_ids(generate_string(embedded_reber_grammar))
                    for _ in range(size // 2)]
    bad_strings = [string_to_ids(generate_corrupted_string(embedded_reber_grammar))
                   for _ in range(size - size // 2)]
    all_strings = good_strings + bad_strings
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array(
            [[1.] for _ in range(len(good_strings))] +
            [[0.] for _ in range(len(bad_strings))]
            )
    return X, y

In [ ]:
np.random.seed(42)

X_train, y_train = generate_dataset(10000)
X_valid, y_valid = generate_dataset(2000)

Let's take a look at the first training sequence:

In [ ]:
X_train[0]

What classes does it belong to?

In [ ]:
y_train[0]

Perfect! We are ready to create the RNN to identify good strings. We build a simple sequence binary classifier:

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

embedding_size = 5

model = keras.models.Sequential(
        [
                keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
                keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embedding_size),
                keras.layers.GRU(30),
                keras.layers.Dense(1, activation="sigmoid")
                ]
        )
optimizer = keras.optimizers.SGD(lr=0.02, momentum=0.95, nesterov=True)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Now let's test our RNN on two tricky strings: the first one is bad while the second one is good. They only differ by the second to last character. If the RNN gets this right, it shows that it managed to notice the pattern that the second letter should always be equal to the second to last letter. That requires a fairly long short-term memory (which is the reason why we used a GRU cell).

In [ ]:
test_strings = ["BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE",
                "BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE"]
X_test = tf.ragged.constant([string_to_ids(s) for s in test_strings], ragged_rank=1)

y_proba = model.predict(X_test)
print()
print("Estimated probability that these are Reber strings:")
for index, string in enumerate(test_strings):
    print("{}: {:.2f}%".format(string, 100 * y_proba[index][0]))

Ta-da! It worked fine. The RNN found the correct answers with very high confidence. :)

## 9.
_Exercise: Train an Encoder–Decoder model that can convert a date string from one format to another (e.g., from "April 22, 2019" to "2019-04-22")._

Let's start by creating the dataset. We will use random days between 1000-01-01 and 9999-12-31:

In [ ]:
from datetime import date

# cannot use strftime()'s %B format since it depends on the locale
MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]


def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    return x, y

Here are a few random dates, displayed in both the input format and the target format:

In [ ]:
np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Let's get the list of all possible characters in the inputs:

In [ ]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS)))) + "01234567890, "
INPUT_CHARS

And here's the list of possible characters in the outputs:

In [ ]:
OUTPUT_CHARS = "0123456789-"

Let's write a function to convert a string to a list of character IDs, as we did in the previous exercise:

In [ ]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

In [ ]:
date_str_to_ids(x_example[0], INPUT_CHARS)

In [ ]:
date_str_to_ids(y_example[0], OUTPUT_CHARS)

In [ ]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor()  # using 0 as the padding token ID


def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

In [ ]:
np.random.seed(42)

X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [ ]:
Y_train[0]

### First version: a very basic seq2seq model

Let's first try the simplest possible model: we feed in the input sequence, which first goes through the encoder (an embedding layer followed by a single LSTM layer), which outputs a vector, then it goes through a decoder (a single LSTM layer, followed by a dense output layer), which outputs a sequence of vectors, each representing the estimated probabilities for all possible output character.

Since the decoder expects a sequence as input, we repeat the vector (which is output by the decoder) as many times as the longest possible output sequence.

In [ ]:
embedding_size = 32
max_output_length = Y_train.shape[1]

np.random.seed(42)
tf.random.set_seed(42)

encoder = keras.models.Sequential(
        [
                keras.layers.Embedding(
                        input_dim=len(INPUT_CHARS) + 1,
                        output_dim=embedding_size,
                        input_shape=[None]
                        ),
                keras.layers.LSTM(128)
                ]
        )

decoder = keras.models.Sequential(
        [
                keras.layers.LSTM(128, return_sequences=True),
                keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")
                ]
        )

model = keras.models.Sequential(
        [
                encoder,
                keras.layers.RepeatVector(max_output_length),
                decoder
                ]
        )

optimizer = keras.optimizers.Nadam()
model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer,
        metrics=["accuracy"]
        )
history = model.fit(
        X_train, Y_train, epochs=20,
        validation_data=(X_valid, Y_valid)
        )

Looks great, we reach 100% validation accuracy! Let's use the model to make some predictions. We will need to be able to convert a sequence of character IDs to a readable string:

In [ ]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("?" + chars)[index] for index in sequence])
            for sequence in ids]

Now we can use the model to convert some dates

In [ ]:
X_new = prepare_date_strs(["September 17, 2009", "July 14, 1789"])

In [ ]:
ids = model.predict_classes(X_new)
for date_str in ids_to_date_strs(ids):
    print(date_str)

Perfect! :)

However, since the model was only trained on input strings of length 18 (which is the length of the longest date), it does not perform well if we try to use it to make predictions on shorter sequences:

In [ ]:
X_new = prepare_date_strs(["May 02, 2020", "July 14, 1789"])

In [ ]:
ids = model.predict_classes(X_new)
for date_str in ids_to_date_strs(ids):
    print(date_str)

Oops! We need to ensure that we always pass sequences of the same length as during training, using padding if necessary. Let's write a little helper function for that:

In [ ]:
max_input_length = X_train.shape[1]


def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X


def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    ids = model.predict_classes(X)
    return ids_to_date_strs(ids)

In [ ]:
convert_date_strs(["May 02, 2020", "July 14, 1789"])

Cool! Granted, there are certainly much easier ways to write a date conversion tool (e.g., using regular expressions or even basic string manipulation), but you have to admit that using neural networks is way cooler. ;-)

However, real-life sequence-to-sequence problems will usually be harder, so for the sake of completeness, let's build a more powerful model.

### Second version: feeding the shifted targets to the decoder (teacher forcing)

Instead of feeding the decoder a simple repetition of the encoder's output vector, we can feed it the target sequence, shifted by one time step to the right. This way, at each time step the decoder will know what the previous target character was. This should help is tackle more complex sequence-to-sequence problems.

Since the first output character of each target sequence has no previous character, we will need a new token to represent the start-of-sequence (sos).

During inference, we won't know the target, so what will we feed the decoder? We can just predict one character at a time, starting with an sos token, then feeding the decoder all the characters that were predicted so far (we will look at this in more details later in this notebook).

But if the decoder's LSTM expects to get the previous target as input at each step, how shall we pass it it the vector output by the encoder? Well, one option is to ignore the output vector, and instead use the encoder's LSTM state as the initial state of the decoder's LSTM (which requires that encoder's LSTM must have the same number of units as the decoder's LSTM).

Now let's create the decoder's inputs (for training, validation and testing). The sos token will be represented using the last possible output character's ID + 1.

In [ ]:
sos_id = len(OUTPUT_CHARS) + 1


def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value=sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1)


X_train_decoder = shifted_output_sequences(Y_train)
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

Let's take a look at the decoder's training inputs:

In [ ]:
X_train_decoder

Now let's build the model. It's not a simple sequential model anymore, so let's use the functional API:

In [ ]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = keras.layers.Embedding(
        input_dim=len(INPUT_CHARS) + 1,
        output_dim=encoder_embedding_size
        )(encoder_input)
_, encoder_state_h, encoder_state_c = keras.layers.LSTM(
        lstm_units, return_state=True
        )(encoder_embedding)
encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = keras.layers.Embedding(
        input_dim=len(OUTPUT_CHARS) + 2,
        output_dim=decoder_embedding_size
        )(decoder_input)
decoder_lstm_output = keras.layers.LSTM(lstm_units, return_sequences=True)(
        decoder_embedding, initial_state=encoder_state
        )
decoder_output = keras.layers.Dense(
        len(OUTPUT_CHARS) + 1,
        activation="softmax"
        )(decoder_lstm_output)

model = keras.models.Model(
        inputs=[encoder_input, decoder_input],
        outputs=[decoder_output]
        )

optimizer = keras.optimizers.Nadam()
model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer,
        metrics=["accuracy"]
        )
history = model.fit(
        [X_train, X_train_decoder], Y_train, epochs=10,
        validation_data=([X_valid, X_valid_decoder], Y_valid)
        )

This model also reaches 100% validation accuracy, but it does so even faster.

Let's once again use the model to make some predictions. This time we need to predict characters one by one.

In [ ]:
sos_id = len(OUTPUT_CHARS) + 1


def predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = tf.fill(dims=(len(X), 1), value=sos_id)
    for index in range(max_output_length):
        pad_size = max_output_length - Y_pred.shape[1]
        X_decoder = tf.pad(Y_pred, [[0, 0], [0, pad_size]])
        Y_probas_next = model.predict([X, X_decoder])[:, index:index + 1]
        Y_pred_next = tf.argmax(Y_probas_next, axis=-1, output_type=tf.int32)
        Y_pred = tf.concat([Y_pred, Y_pred_next], axis=1)
    return ids_to_date_strs(Y_pred[:, 1:])

In [ ]:
predict_date_strs(["July 14, 1789", "May 01, 2020"])

Works fine! :)

### Third version: using TF-Addons's seq2seq implementation

Let's build exactly the same model, but using TF-Addon's seq2seq API. The implementation below is almost very similar to the TFA example higher in this notebook, except without the model input to specify the output sequence length, for simplicity (but you can easily add it back in if you need it for your projects, when the output sequences have very different lengths).

In [ ]:
import tensorflow_addons as tfa

np.random.seed(42)
tf.random.set_seed(42)

encoder_embedding_size = 32
decoder_embedding_size = 32
units = 128

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

encoder_embeddings = keras.layers.Embedding(
        len(INPUT_CHARS) + 1, encoder_embedding_size
        )(encoder_inputs)

decoder_embedding_layer = keras.layers.Embedding(
        len(INPUT_CHARS) + 2, decoder_embedding_size
        )
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

encoder = keras.layers.LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(units)
output_layer = keras.layers.Dense(len(OUTPUT_CHARS) + 1)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
        decoder_cell,
        sampler,
        output_layer=output_layer
        )
final_outputs, final_state, final_sequence_lengths = decoder(
        decoder_embeddings,
        initial_state=encoder_state
        )
Y_proba = keras.layers.Activation("softmax")(final_outputs.rnn_output)

model = keras.models.Model(
        inputs=[encoder_inputs, decoder_inputs],
        outputs=[Y_proba]
        )
optimizer = keras.optimizers.Nadam()
model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer,
        metrics=["accuracy"]
        )
history = model.fit(
        [X_train, X_train_decoder], Y_train, epochs=15,
        validation_data=([X_valid, X_valid_decoder], Y_valid)
        )

And once again, 100% validation accuracy! To use the model, we can just reuse the `predict_date_strs()` function:

In [ ]:
predict_date_strs(["July 14, 1789", "May 01, 2020"])

However, there's a much more efficient way to perform inference. Until now, during inference, we've run the model once for each new character. Instead, we can create a new decoder, based on the previously trained layers, but using a `GreedyEmbeddingSampler` instead of a `TrainingSampler`.

At each time step, the `GreedyEmbeddingSampler` will compute the argmax of the decoder's outputs, and run the resulting token IDs through the decoder's embedding layer. Then it will feed the resulting embeddings to the decoder's LSTM cell at the next time step. This way, we only need to run the decoder once to get the full prediction.

In [ ]:
inference_sampler = tfa.seq2seq.sampler.GreedyEmbeddingSampler(
        embedding_fn=decoder_embedding_layer
        )
inference_decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
        decoder_cell, inference_sampler, output_layer=output_layer,
        maximum_iterations=max_output_length
        )
batch_size = tf.shape(encoder_inputs)[:1]
start_tokens = tf.fill(dims=batch_size, value=sos_id)
final_outputs, final_state, final_sequence_lengths = inference_decoder(
        start_tokens,
        initial_state=encoder_state,
        start_tokens=start_tokens,
        end_token=0
        )

inference_model = keras.models.Model(
        inputs=[encoder_inputs],
        outputs=[final_outputs.sample_id]
        )

A few notes:
* The `GreedyEmbeddingSampler` needs the `start_tokens` (a vector containing the start-of-sequence ID for each decoder sequence), and the `end_token` (the decoder will stop decoding a sequence once the model outputs this token).
* We must set `maximum_iterations` when creating the `BasicDecoder`, or else it may run into an infinite loop (if the model never outputs the end token for at least one of the sequences). This would force you would to restart the Jupyter kernel.
* The decoder inputs are not needed anymore, since all the decoder inputs are generated dynamically based on the outputs from the previous time step.
* The model's outputs are `final_outputs.sample_id` instead of the softmax of `final_outputs.rnn_outputs`. This allows us to directly get the argmax of the model's outputs. If you prefer to have access to the logits, you can replace `final_outputs.sample_id` with `final_outputs.rnn_outputs`.

Now we can write a simple function that uses the model to perform the date format conversion:

In [ ]:
def fast_predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = inference_model.predict(X)
    return ids_to_date_strs(Y_pred)

In [ ]:
fast_predict_date_strs(["July 14, 1789", "May 01, 2020"])

Let's check that it really is faster:

In [ ]:
%timeit predict_date_strs(["July 14, 1789", "May 01, 2020"])

In [ ]:
%timeit fast_predict_date_strs(["July 14, 1789", "May 01, 2020"])

That's more than a 10x speedup! And it would be even more if we were handling longer sequences.

### Fourth version: using TF-Addons's seq2seq implementation with a scheduled sampler

**Warning**: due to a TF bug, this version only works using TensorFlow 2.2.

When we trained the previous model, at each time step _t_ we gave the model the target token for time step _t_ - 1. However, at inference time, the model did not get the previous target at each time step. Instead, it got the previous prediction. So there is a discrepancy between training and inference, which may lead to disappointing performance. To alleviate this, we can gradually replace the targets with the predictions, during training. For this, we just need to replace the `TrainingSampler` with a `ScheduledEmbeddingTrainingSampler`, and use a Keras callback to gradually increase the `sampling_probability` (i.e., the probability that the decoder will use the prediction from the previous time step rather than the target for the previous time step).

In [ ]:
import tensorflow_addons as tfa

np.random.seed(42)
tf.random.set_seed(42)

n_epochs = 20
encoder_embedding_size = 32
decoder_embedding_size = 32
units = 128

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

encoder_embeddings = keras.layers.Embedding(
        len(INPUT_CHARS) + 1, encoder_embedding_size
        )(encoder_inputs)

decoder_embedding_layer = keras.layers.Embedding(
        len(INPUT_CHARS) + 2, decoder_embedding_size
        )
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

encoder = keras.layers.LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.ScheduledEmbeddingTrainingSampler(
        sampling_probability=0.,
        embedding_fn=decoder_embedding_layer
        )
# we must set the sampling_probability after creating the sampler
# (see https://github.com/tensorflow/addons/pull/1714)
sampler.sampling_probability = tf.Variable(0.)

decoder_cell = keras.layers.LSTMCell(units)
output_layer = keras.layers.Dense(len(OUTPUT_CHARS) + 1)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
        decoder_cell,
        sampler,
        output_layer=output_layer
        )
final_outputs, final_state, final_sequence_lengths = decoder(
        decoder_embeddings,
        initial_state=encoder_state
        )
Y_proba = keras.layers.Activation("softmax")(final_outputs.rnn_output)

model = keras.models.Model(
        inputs=[encoder_inputs, decoder_inputs],
        outputs=[Y_proba]
        )
optimizer = keras.optimizers.Nadam()
model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer,
        metrics=["accuracy"]
        )


def update_sampling_probability(epoch, logs):
    proba = min(1.0, epoch / (n_epochs - 10))
    sampler.sampling_probability.assign(proba)


sampling_probability_cb = keras.callbacks.LambdaCallback(
        on_epoch_begin=update_sampling_probability
        )
history = model.fit(
        [X_train, X_train_decoder], Y_train, epochs=n_epochs,
        validation_data=([X_valid, X_valid_decoder], Y_valid),
        callbacks=[sampling_probability_cb]
        )

Not quite 100% validation accuracy, but close enough!

For inference, we could do the exact same thing as earlier, using a `GreedyEmbeddingSampler`. However, just for the sake of completeness, let's use a `SampleEmbeddingSampler` instead. It's almost the same thing, except that instead of using the argmax of the model's output to find the token ID, it treats the outputs as logits and uses them to sample a token ID randomly. This can be useful when you want to generate text. The `softmax_temperature` argument serves the
same purpose as when we generated Shakespeare-like text (the higher this argument, the more random the generated text will be).

In [ ]:
softmax_temperature = tf.Variable(1.)

inference_sampler = tfa.seq2seq.sampler.SampleEmbeddingSampler(
        embedding_fn=decoder_embedding_layer,
        softmax_temperature=softmax_temperature
        )
inference_decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
        decoder_cell, inference_sampler, output_layer=output_layer,
        maximum_iterations=max_output_length
        )
batch_size = tf.shape(encoder_inputs)[:1]
start_tokens = tf.fill(dims=batch_size, value=sos_id)
final_outputs, final_state, final_sequence_lengths = inference_decoder(
        start_tokens,
        initial_state=encoder_state,
        start_tokens=start_tokens,
        end_token=0
        )

inference_model = keras.models.Model(
        inputs=[encoder_inputs],
        outputs=[final_outputs.sample_id]
        )

In [ ]:
def creative_predict_date_strs(date_strs, temperature=1.0):
    softmax_temperature.assign(temperature)
    X = prepare_date_strs_padded(date_strs)
    Y_pred = inference_model.predict(X)
    return ids_to_date_strs(Y_pred)

In [ ]:
tf.random.set_seed(42)

creative_predict_date_strs(["July 14, 1789", "May 01, 2020"])

Dates look good at room temperature. Now let's heat things up a bit:

In [ ]:
tf.random.set_seed(42)

creative_predict_date_strs(
        ["July 14, 1789", "May 01, 2020"],
        temperature=5.
        )

Oops, the dates are overcooked, now. Let's call them "creative" dates.

### Fifth version: using TFA seq2seq, the Keras subclassing API and attention mechanisms

The sequences in this problem are pretty short, but if we wanted to tackle longer sequences, we would probably have to use attention mechanisms. While it's possible to code our own implementation, it's simpler and more efficient to use TF-Addons's implementation instead. Let's do that now, this time using Keras' subclassing API.

**Warning**: due to a TensorFlow bug (see [this issue](https://github.com/tensorflow/addons/issues/1153) for details), the `get_initial_state()` method fails in eager mode, so for now we have to use the subclassing API, as Keras automatically calls `tf.function()` on the `call()` method (so it runs in graph mode).

In this implementation, we've reverted back to using the `TrainingSampler`, for simplicity (but you can easily tweak it to use a `ScheduledEmbeddingTrainingSampler` instead). We also use a `GreedyEmbeddingSampler` during inference, so this class is pretty easy to use:

In [ ]:
class DateTranslation(keras.models.Model):
    def __init__(self, units=128, encoder_embedding_size=32,
                 decoder_embedding_size=32, **kwargs
                 ):
        super().__init__(**kwargs)
        self.encoder_embedding = keras.layers.Embedding(
                input_dim=len(INPUT_CHARS) + 1,
                output_dim=encoder_embedding_size
                )
        self.encoder = keras.layers.LSTM(
                units,
                return_sequences=True,
                return_state=True
                )
        self.decoder_embedding = keras.layers.Embedding(
                input_dim=len(OUTPUT_CHARS) + 2,
                output_dim=decoder_embedding_size
                )
        self.attention = tfa.seq2seq.LuongAttention(units)
        decoder_inner_cell = keras.layers.LSTMCell(units)
        self.decoder_cell = tfa.seq2seq.AttentionWrapper(
                cell=decoder_inner_cell,
                attention_mechanism=self.attention
                )
        output_layer = keras.layers.Dense(len(OUTPUT_CHARS) + 1)
        self.decoder = tfa.seq2seq.BasicDecoder(
                cell=self.decoder_cell,
                sampler=tfa.seq2seq.sampler.TrainingSampler(),
                output_layer=output_layer
                )
        self.inference_decoder = tfa.seq2seq.BasicDecoder(
                cell=self.decoder_cell,
                sampler=tfa.seq2seq.sampler.GreedyEmbeddingSampler(
                        embedding_fn=self.decoder_embedding
                        ),
                output_layer=output_layer,
                maximum_iterations=max_output_length
                )

    def call(self, inputs, training=None):
        encoder_input, decoder_input = inputs
        encoder_embeddings = self.encoder_embedding(encoder_input)
        encoder_outputs, encoder_state_h, encoder_state_c = self.encoder(
                encoder_embeddings,
                training=training
                )
        encoder_state = [encoder_state_h, encoder_state_c]

        self.attention(
                encoder_outputs,
                setup_memory=True
                )

        decoder_embeddings = self.decoder_embedding(decoder_input)

        decoder_initial_state = self.decoder_cell.get_initial_state(
                decoder_embeddings
                )
        decoder_initial_state = decoder_initial_state.clone(
                cell_state=encoder_state
                )

        if training:
            decoder_outputs, _, _ = self.decoder(
                    decoder_embeddings,
                    initial_state=decoder_initial_state,
                    training=training
                    )
        else:
            start_tokens = tf.zeros_like(encoder_input[:, 0]) + sos_id
            decoder_outputs, _, _ = self.inference_decoder(
                    decoder_embeddings,
                    initial_state=decoder_initial_state,
                    start_tokens=start_tokens,
                    end_token=0
                    )

        return tf.nn.softmax(decoder_outputs.rnn_output)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = DateTranslation()
optimizer = keras.optimizers.Nadam()
model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer,
        metrics=["accuracy"]
        )
history = model.fit(
        [X_train, X_train_decoder], Y_train, epochs=25,
        validation_data=([X_valid, X_valid_decoder], Y_valid)
        )

Not quite 100% validation accuracy, but close. It took a bit longer to converge this time, but there were also more parameters and more computations per iteration. And we did not use a scheduled sampler.

To use the model, we can write yet another little function:

In [ ]:
def fast_predict_date_strs_v2(date_strs):
    X = prepare_date_strs_padded(date_strs)
    X_decoder = tf.zeros(shape=(len(X), max_output_length), dtype=tf.int32)
    Y_probas = model.predict([X, X_decoder])
    Y_pred = tf.argmax(Y_probas, axis=-1)
    return ids_to_date_strs(Y_pred)

In [ ]:
fast_predict_date_strs_v2(["July 14, 1789", "May 01, 2020"])

There are still a few interesting features from TF-Addons that you may want to look at:
* Using a `BeamSearchDecoder` rather than a `BasicDecoder` for inference. Instead of outputing the character with the highest probability, this decoder keeps track of the several candidates, and keeps only the most likely sequences of candidates (see chapter 16 in the book for more details).
* Setting masks or specifying `sequence_length` if the input or target sequences may have very different lengths.
* Using a `ScheduledOutputTrainingSampler`, which gives you more flexibility than the `ScheduledEmbeddingTrainingSampler` to decide how to feed the output at time _t_ to the cell at time _t_+1. By default it feeds the outputs directly to cell, without computing the argmax ID and passing it through an embedding layer. Alternatively, you specify a `next_inputs_fn` function that will be used to convert the cell outputs to inputs at the next step.

## 10.
_Exercise: Go through TensorFlow's [Neural Machine Translation with Attention tutorial](https://homl.info/nmttuto)._

Simply open the Colab and follow its instructions. Alternatively, if you want a simpler example of using TF-Addons's seq2seq implementation for Neural Machine Translation (NMT), look at the solution to the previous question. The last model implementation will give you a simpler example of using TF-Addons to build an NMT model using attention mechanisms.

## 11.
_Exercise: Use one of the recent language models (e.g., GPT) to generate more convincing Shakespearean text._

The simplest way to use recent language models is to use the excellent [transformers library](https://huggingface.co/transformers/), open sourced by Hugging Face. It provides many modern neural net architectures (including BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet and more) for Natural Language Processing (NLP), including many pretrained models. It relies on either TensorFlow or PyTorch. Best of all: it's amazingly simple to use.

First, let's load a pretrained model. In this example, we will use OpenAI's GPT model, with an additional Language Model on top (just a linear layer with weights tied to the input embeddings). Let's import it and load the pretrained weights (this will download about 445MB of data to `~/.cache/torch/transformers`):

In [ ]:
from transformers import TFOpenAIGPTLMHeadModel

model = TFOpenAIGPTLMHeadModel.from_pretrained("openai-gpt")

Next we will need a specialized tokenizer for this model. This one will try to use the [spaCy](https://spacy.io/) and [ftfy](https://pypi.org/project/ftfy/) libraries if they are installed, or else it will fall back to BERT's `BasicTokenizer` followed by Byte-Pair Encoding (which should be fine for most use cases).

In [ ]:
from transformers import OpenAIGPTTokenizer

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")

Now let's use the tokenizer to tokenize and encode the prompt text:

In [ ]:
prompt_text = "This royal throne of kings, this sceptred isle"
encoded_prompt = tokenizer.encode(
        prompt_text,
        add_special_tokens=False,
        return_tensors="tf"
        )
encoded_prompt

Easy! Next, let's use the model to generate text after the prompt. We will generate 5 different sentences, each starting with the prompt text, followed by 40 additional tokens. For an explanation of what all the hyperparameters do, make sure to check out this great [blog post](https://huggingface.co/blog/how-to-generate) by Patrick von Platen (from Hugging Face). You can play around with the hyperparameters to try to obtain better results.

In [ ]:
num_sequences = 5
length = 40

generated_sequences = model.generate(
        input_ids=encoded_prompt,
        do_sample=True,
        max_length=length + len(encoded_prompt[0]),
        temperature=1.0,
        top_k=0,
        top_p=0.9,
        repetition_penalty=1.0,
        num_return_sequences=num_sequences,
        )

generated_sequences

Now let's decode the generated sequences and print them:

In [ ]:
for sequence in generated_sequences:
    text = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
    print(text)
    print("-" * 80)

You can try more recent (and larger) models, such as GPT-2, CTRL, Transformer-XL or XLNet, which are all available as pretrained models in the transformers library, including variants with Language Models on top. The preprocessing steps vary slightly between models, so make sure to check out this [generation example](https://github.com/huggingface/transformers/blob/master/examples/run_generation.py) from the transformers documentation (this example uses PyTorch, but it will work with very little tweaks, such as adding `TF` at the beginning of the model class name, removing the `.to()` method calls, and using `return_tensors="tf"` instead of `"pt"`.

Hope you enjoyed this chapter! :)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
for i in range(3):
    print(loss_object(i, [0.1, 0.7, 0.2]))

In [ ]:
a = np.array([1, 0, 2])
b = np.zeros((a.size, a.max() + 1))
b[np.arange(a.size), a] = 1
print(b)
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
for i in range(3):
    print(loss_object(b[i], [0.1, 0.7, 0.2]))

In [ ]:
ll = np.arange(10)
print(ll)
print(ll[:-1])
print(ll[1:])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

n_steps = 5
ds = tf.data.Dataset.from_tensor_slices(tf.range(15))
for data in ds:
    print(data)

In [ ]:
ds = ds.window(n_steps, shift=1, drop_remainder=True)
for data in ds:
    print(list(data.as_numpy_iterator()))

In [ ]:
ds = ds.flat_map(lambda window: window.batch(n_steps))

In [ ]:
for data in ds:
    print(data)

In [ ]:
#ds = ds.shuffle(10).map(lambda window: (window[:-1], window[1:]))
ds = ds.map(lambda window: (window[:-1], window[1:]))

In [ ]:
for X, y in ds:
    print(X, y)

In [ ]:
ds = ds.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(ds):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())


In [67]:
l = [1, 2, 3]


def bubble_sort(list):
    pass


bubble_sort(l)


1
2
3


[1, 4, 9]

In [70]:
def fun(a, b, c=0):
    print(a, b, c)
    return a, b, c


x, y, z = fun(1, 2)

1 2 0


In [5]:

def maximum(list):
    max = list[0]
    for i in range(1, len(list)):
        if list[i] > max:
            max = list[i]
    return max


maximum([1, 2, 3, 4, 1])

4